In [ ]:
import torch

'''
import torch.optim as optim
import pandas as pd
from matplotlib import pyplot as plt
from torch.utils import data
from torch import nn
import time
from IPython import display
from torch.nn import functional as F

# load and preprocess data
source = pd.read_csv("../input/facial-expression-recognitionferchallenge/fer2013/fer2013/fer2013.csv")
processed_data = []
for i in range(0, len(source)):
    item = source.iloc[i, 1]
    temp1 = [float(j) for j in item.split(" ")]
    if not any(temp1):
        continue
    temp2 = []
    for j in range(0, 48):
        temp2.append(temp1[48*j: 48*j+48])
    temp3 = [torch.tensor([temp2]), source.iloc[i, 0]]
    processed_data.append(temp3)

lr, num_epochs, batch_size = 0.001, 100, 64
train_iter = data.DataLoader(processed_data[:int(len(processed_data) * 0.9)], batch_size, shuffle=True)
test_iter = data.DataLoader(processed_data[int(len(processed_data) * 0.9):], batch_size, shuffle=False)

transfer_pattern = ["angry", "disgust", "fear", "happy", "sad", "surprise", "neutral"]


def plot(series):
    plt.title(transfer_pattern[processed_data[series][1]])
    plt.imshow(processed_data[series][0], cmap="gray")
    plt.show()

# train
argmax = lambda x, *args, **kwargs: x.argmax(*args, **kwargs)
astype = lambda x, *args, **kwargs: x.type(*args, **kwargs)
reduce_sum = lambda x, *args, **kwargs: x.sum(*args, **kwargs)
size = lambda x, *args, **kwargs: x.numel(*args, **kwargs)


class Timer:
    def __init__(self):
        self.times = []
        self.start()

    def start(self):
        self.tik = time.time()

    def stop(self):
        self.times.append(time.time() - self.tik)
        return self.times[-1]

    def sum(self):
        return sum(self.times)


class Accumulator:
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def __getitem__(self, idx):
        return self.data[idx]


def set_axes(axes, xlabel, ylabel, xlim, ylim, xscale, yscale, legend):
    axes.set_xlabel(xlabel)
    axes.set_ylabel(ylabel)
    axes.set_xscale(xscale)
    axes.set_yscale(yscale)
    axes.set_xlim(xlim)
    axes.set_ylim(ylim)
    if legend:
        axes.legend(legend)
    axes.grid()


class Animator:
    def __init__(self, xlabel=None, ylabel=None, legend=None, xlim=None,
                 ylim=None, xscale='linear', yscale='linear',
                 fmts=('-', 'm--', 'g-.', 'r:'), nrows=1, ncols=1,
                 figsize=(3.5, 2.5)):
        if legend is None:
            legend = []
        display.set_matplotlib_formats('svg')
        self.fig, self.axes = plt.subplots(nrows, ncols, figsize=figsize)
        if nrows * ncols == 1:
            self.axes = [self.axes, ]
        self.config_axes = lambda: set_axes(
            self.axes[0], xlabel, ylabel, xlim, ylim, xscale, yscale, legend)
        self.X, self.Y, self.fmts = None, None, fmts

    def add(self, x, y):
        if not hasattr(y, "__len__"):
            y = [y]
        n = len(y)
        if not hasattr(x, "__len__"):
            x = [x] * n
        if not self.X:
            self.X = [[] for _ in range(n)]
        if not self.Y:
            self.Y = [[] for _ in range(n)]
        for i, (a, b) in enumerate(zip(x, y)):
            if a is not None and b is not None:
                self.X[i].append(a)
                self.Y[i].append(b)
        self.axes[0].cla()
        for x, y, fmt in zip(self.X, self.Y, self.fmts):
            self.axes[0].plot(x, y, fmt)
        self.config_axes()
        display.clear_output(wait=True)


def try_gpu(i=0):
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')


def accuracy(y_hat, y):
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = argmax(y_hat, axis=1)
    cmp = astype(y_hat, y.dtype) == y
    return float(reduce_sum(astype(cmp, y.dtype)))


def evaluate_accuracy_gpu(net, data_iter, device=None):
    net.eval()
    if not device:
        device = next(iter(net.parameters())).device
    metric = Accumulator(2)
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        metric.add(accuracy(net(X), y), size(y))
    return metric[0] / metric[1]

'''
class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                e_w = p.grad * scale
                p.add_(e_w)  # climb to the local maximum "w + e(w)"
                self.state[p]["e_w"] = e_w

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.sub_(self.state[p]["e_w"])  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    def step(self, closure=None):
        raise NotImplementedError("SAM doesn't work like the other optimizers, you should first call `first_step` and the `second_step`; see the documentation for more info.")

    def _grad_norm(self):
        norm = torch.norm(
                    torch.stack([
                        p.grad.norm(p=2)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm

'''
def train_plot(net, train_iter, test_iter, num_epochs, lr, optimizer, scheduler, 
              device=try_gpu()):
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            torch.nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    print('training on', device)
    net.to(device)
    loss = nn.CrossEntropyLoss()
    animator = Animator(xlabel='epoch', xlim=[0, num_epochs],
                            legend=['loss', 'train acc', 'test acc'])
    timer = Timer()
    for epoch in range(num_epochs):
        metric = Accumulator(3)
        scheduler.step()
        for i, (X, y) in enumerate(train_iter):
            timer.start()
            net.train()
            #optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = F.cross_entropy(net(X), y)
            l.backward()
            optimizer.first_step(zero_grad=True)
            F.cross_entropy(net(X), y).backward()
            optimizer.second_step(zero_grad=True)
            
            with torch.no_grad():
                metric.add(l * X.shape[0], accuracy(y_hat, y), X.shape[0])
            timer.stop()
            train_loss = metric[0]/metric[2]
            train_acc = metric[1]/metric[2]

            if (i + 1) % 50 == 0:
                animator.add(epoch + i / len(train_iter),
                             (train_loss, train_acc, None))
        test_acc = evaluate_accuracy_gpu(net, test_iter)
        animator.add(epoch+1, (None, None, test_acc))
        print("Epoch: {}/{}, train accuray: {}, test accuracy: {}".format(epoch, num_epochs, train_acc, test_acc))
    print(f'loss {train_loss:.3f}, train acc {train_acc:.3f}, '
          f'test acc {test_acc:.3f}')
    print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec '
          f'on {str(device)}')

    
class Mish(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        return x * torch.tanh(F.softplus(x))
'''
    
# construct net
'''
stage1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=3, padding=1), Mish(), nn.BatchNorm2d(64),
                       nn.Conv2d(64, 64, kernel_size=3, padding=1), Mish(), nn.BatchNorm2d(64),
                       nn.Conv2d(64, 64, kernel_size=3, padding=1), Mish(), nn.BatchNorm2d(64),
                       nn.MaxPool2d(kernel_size=2), nn.Dropout(p=0.25))

stage2 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=3, padding=1), Mish(), nn.BatchNorm2d(128),
                       nn.Conv2d(128, 128, kernel_size=3, padding=1), Mish(), nn.BatchNorm2d(128),
                       nn.Conv2d(128, 128, kernel_size=3, padding=1), Mish(), nn.BatchNorm2d(128),
                       nn.MaxPool2d(kernel_size=2), nn.Dropout(p=0.25))

stage3 = nn.Sequential(nn.Conv2d(128, 128, kernel_size=3, padding=1), Mish(), nn.BatchNorm2d(128),
                       nn.Conv2d(128, 128, kernel_size=3, padding=1), Mish(), nn.BatchNorm2d(128),
                       nn.Conv2d(128, 128, kernel_size=3, padding=1), Mish(), nn.BatchNorm2d(128),
                       nn.MaxPool2d(kernel_size=2), nn.Dropout(p=0.25))

stage4 = nn.Sequential(nn.Conv2d(128, 128, kernel_size=3, padding=1), Mish(), nn.BatchNorm2d(128),
                       nn.Conv2d(128, 128, kernel_size=3, padding=1), Mish(), nn.BatchNorm2d(128),
                       nn.Conv2d(128, 128, kernel_size=3, padding=1), Mish(), nn.BatchNorm2d(128),
                       nn.MaxPool2d(kernel_size=2), nn.Dropout(p=0.25))

stage5 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=3, padding=1), Mish(), nn.BatchNorm2d(256),
                       nn.Conv2d(256, 256, kernel_size=3, padding=1), Mish(), nn.BatchNorm2d(256),
                       nn.Conv2d(256, 256, kernel_size=3, padding=1), Mish(), nn.BatchNorm2d(256),
                       nn.MaxPool2d(kernel_size=2), nn.Dropout(p=0.25))

net = nn.Sequential(stage1, stage2, stage3, stage4, stage5, nn.Flatten(), nn.Linear(256, 7))
'''

'''
import torchvision.models as md
net = md.wide_resnet101_2()
net.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
net.fc = nn.Linear(2048, 7)
'''

'''
X = torch.rand(size=(1, 1, 32, 32))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)
'''

'''
base_optimizer = optim.SGD
optimizer = SAM(net.parameters(), base_optimizer, lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20, 32], gamma=0.1)

train_plot(net, train_iter, test_iter, num_epochs, lr, optimizer, scheduler)
plt.savefig("Result.jpg")
plt.show()
torch.save(net.state_dict(), "Net.param")
'''

In [ ]:
import math
import numpy as np
import torchvision.models as models
import torch.utils.data as data
from torchvision import transforms
import cv2
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import pandas as pd
import os ,torch
import torch.nn as nn

    
class RafDataSet(data.Dataset):
    def __init__(self, dt, lb):
        self.data = dt
        self.label = lb

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx][0]
        label = self.label[idx]
        
        return image, label, idx

'''
class Res18Feature(nn.Module):
    def __init__(self, pretrained = True, num_classes = 7, drop_rate = 0):
        super(Res18Feature, self).__init__()
        self.drop_rate = drop_rate
        
        #resnet  = models.resnet18(pretrained)
        #resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        #self.features = nn.Sequential(*list(resnet.children())[:-1]) # after avgpool 512x1
        #fc_in_dim = list(resnet.children())[-1].in_features # original fc layer's in dimention 512
        
        
        #vgg19_bn
        
        resnet = models.vgg13_bn(pretrained)
        resnet.features[0] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        #self.features = nn.Sequential(*list(resnet.children())[:-1], *list(resnet.classifier[:-1]))
        self.features = nn.Sequential(*list(resnet.children())[:-1])
        self.nxt = resnet.classifier[:-1]
        
        fc_in_dim = resnet.classifier[-1].in_features
        
        
    
        self.fc = nn.Linear(fc_in_dim, num_classes) # new fc layer 512x7
        self.alpha = nn.Sequential(nn.Linear(fc_in_dim, 1),nn.Sigmoid())
        
        

    def forward(self, x):
        x = self.features(x)
        
        #x = self.nxt(x)
        
        if self.drop_rate > 0:
            x =  nn.Dropout(self.drop_rate)(x)
        x = x.view(x.size(0), -1)
        x = self.nxt(x)
        
        attention_weights = self.alpha(x)
        out = attention_weights * self.fc(x)
        return attention_weights, out
'''

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride = stride, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride = stride, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu2 = nn.ReLU()
        self.skip = (in_channels==out_channels)

    def forward(self, x):
        identity = x
        out = self.bn1(x)
        out = self.relu1(out)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.relu2(out)
        out = self.conv2(out)
        if self.skip :
            out += identity

        return out

class Res18Feature(nn.Module):
    def __init__(self, pretrained=False, num_classes=7, drop_rate=0):
        super(Res18Feature, self).__init__()
        self.drop_rate = drop_rate
        self.convin = nn.Conv2d(1, 64, kernel_size=5, stride=2, padding=2)
        self.bnin = nn.BatchNorm2d(64, track_running_stats=True)
        self.relu = nn.ReLU()
        self.stack1 = BasicBlock(64,64,1)
        self.stack2 = BasicBlock(64,128,1)
        self.stack3 = BasicBlock(128,256,1)
        #self.stack4 = BasicBlock(256,512,1)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.alpha = nn.Sequential(nn.Linear(256, 1), nn.Sigmoid())
        self.fc = nn.Linear(256,7)

    def forward(self, x):
        x = self.convin(x)
        x = self.bnin(x)
        x = self.relu(x)

        x = self.stack1(x)
        x = self.stack2(x)
        x = self.stack3(x)
        #x = self.stack4(x)

        x = self.avgpool(x)
        if self.drop_rate > 0:
            x = nn.Dropout(self.drop_rate)(x)
        x = x.view(x.size(0),-1)

        attention_weights = self.alpha(x)
        x = attention_weights * self.fc(x)

        return attention_weights, x
        
def run_training():

    imagenet_pretrained = False
    res18 = Res18Feature(pretrained = imagenet_pretrained, drop_rate = 0.2) 
    
    source = pd.read_csv("../input/facial-expression-recognitionferchallenge/fer2013/fer2013/fer2013.csv")
    trans = transforms.Compose([transforms.ToPILImage(), transforms.Resize(128), transforms.ToTensor()])
    processed_data = []
    for i in range(0, len(source)):
        item = source.iloc[i, 1]
        temp1 = np.array([int(j) for j in item.split(" ")])
        temp2 = np.reshape(temp1, (48,48))

        temp3 = [trans(np.uint8(temp2)) * 255, source.iloc[i, 0]]
        processed_data.append(temp3)

    lb = np.array(source.iloc[:,0]).reshape(-1)
    lr, num_epochs, batch_size = 0.001, 100, 64
    train_loader = data.DataLoader(RafDataSet(processed_data[:int(len(processed_data) * 0.9)], lb), batch_size, shuffle=True)
    val_loader = data.DataLoader(RafDataSet(processed_data[int(len(processed_data) * 0.9):], lb[int(len(processed_data) * 0.9):]), batch_size, shuffle=False)
    
    params = res18.parameters()
    
    
    optimizer = torch.optim.SGD(params, 0.05,
                                    momentum=0.8,
                                    weight_decay = 1e-4)
    
    
    #base_optimizer = torch.optim.SGD
    #optimizer = SAM(params, base_optimizer, lr=0.1, momentum=0.9, weight_decay = 1e-4)

    
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.9)
    res18 = res18.cuda()
    criterion = torch.nn.CrossEntropyLoss()
    
    margin_1 = 0.07
    margin_2 = 0.2
    beta = 0.7
    
    print(next(iter(train_loader)))
    
    
    for i in range(1, 50 + 1):
        running_loss = 0.0
        correct_sum = 0
        iter_cnt = 0
        res18.train()
        for batch_i, (imgs, targets, indexes) in enumerate(train_loader):
            batch_sz = imgs.size(0) 
            iter_cnt += 1
            tops = int(batch_sz* beta)
            optimizer.zero_grad()
            imgs = imgs.cuda()
            targets = targets.cuda()
            
            #add for SAM
            
            #_, ot = res18(imgs)
            #criterion(ot, targets).backward()
            #optimizer.first_step(zero_grad=True)
            
            
            attention_weights, outputs = res18(imgs)
            
            # Rank Regularization
            _, top_idx = torch.topk(attention_weights.squeeze(), tops)
            _, down_idx = torch.topk(attention_weights.squeeze(), batch_sz - tops, largest = False)

            high_group = attention_weights[top_idx]
            low_group = attention_weights[down_idx]
            high_mean = torch.mean(high_group)
            low_mean = torch.mean(low_group)
            diff  = low_mean - high_mean + margin_1

            if diff > 0:
                RR_loss = diff
            else:
                RR_loss = 0.0
            
            
            loss = criterion(outputs, targets) + RR_loss 
            loss.backward()
            #add for SAM
            optimizer.step()
            #optimizer.second_step(zero_grad=True)
            
            running_loss += loss
            _, predicts = torch.max(outputs, 1)
            correct_num = torch.eq(predicts, targets).sum()
            correct_sum += correct_num

            # Relabel samples
            if i >= 12:
                sm = torch.softmax(outputs, dim = 1)
                Pmax, predicted_labels = torch.max(sm, 1) # predictions
                Pgt = torch.gather(sm, 1, targets.view(-1,1)).squeeze() # retrieve predicted probabilities of targets
                true_or_false = Pmax - Pgt > margin_2
                update_idx = true_or_false.nonzero().squeeze() # get samples' index in this mini-batch where (Pmax - Pgt > margin_2)
                label_idx = indexes[update_idx] # get samples' index in train_loader
                relabels = predicted_labels[update_idx] # predictions where (Pmax - Pgt > margin_2)
                train_loader.dataset.label[label_idx.cpu().numpy()] = relabels.cpu().numpy() # relabel samples in train_loader
                
        scheduler.step()
        acc = correct_sum.float() / float(len(processed_data)*0.9)
        running_loss = running_loss/iter_cnt
        print('[Epoch %d] Training accuracy: %.4f. Loss: %.3f' % (i, acc, running_loss))
        
        with torch.no_grad():
            running_loss = 0.0
            iter_cnt = 0
            bingo_cnt = 0
            sample_cnt = 0
            res18.eval()
            for batch_i, (imgs, targets, _) in enumerate(val_loader):
                _, outputs = res18(imgs.cuda())
                targets = targets.cuda()
                loss = criterion(outputs, targets)
                running_loss += loss
                iter_cnt+=1
                _, predicts = torch.max(outputs, 1)
                correct_num  = torch.eq(predicts,targets)
                bingo_cnt += correct_num.sum().cpu()
                sample_cnt += outputs.size(0)
                
            running_loss = running_loss/iter_cnt   
            acc = bingo_cnt.float()/float(sample_cnt)
            acc = np.around(acc.numpy(),4)
            print("[Epoch %d] Validation accuracy:%.4f. Loss:%.3f" % (i, acc, running_loss))
           
            if acc > 0.7 :
                torch.save({'iter': i,
                            'model_state_dict': res18.state_dict(),
                             'optimizer_state_dict': optimizer.state_dict(),},
                            os.path.join('models', "epoch"+str(i)+"_acc"+str(acc)+".pth"))
                print('Model saved.')
    
     
            
               
run_training()